In [8]:
import os, json, sys
import pandas as pd
from sqlalchemy import create_engine
from PIL import Image
#from time import time
import utils.database as db
import utils.corpora as corp
import utils.fingerprints as finger

import operator
from scipy.sparse import csr_matrix
from sparse_som import *
import numpy as np
import re
import conf.conn as cfg
#from tqdm import tnrange, tqdm_notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h3>Insert snippets into database</h3>

In [11]:
#set wikipedia folder where dump json files are located
try:
    wikifilesdir = "F:/RESEARCH/TESE/corpora/wikifiles/01012018/json"
    os.chdir(wikifilesdir)
except:
    print('ERROR: ' +str(sys.exc_info()[0]))

#register file system json files in DB
def register():
    i = 0
    for subdir, dirs, files in os.walk(wikifilesdir):
    
        for file in files:
            if i % 5 == 0:
                print (str(i)+ ' files processed')
            #print (os.path.join(subdir, file))
            filepath = subdir + os.sep + file
            path = filepath.split(os.sep)[-3:]
            path = '/'.join(path)
            db.register_file(cfg, path)
            i = i + 1

    print ("DONE. %s files registered "% (i))

#process file system json files in DB and move file to processed destination folder
def process():
    limit = 100
    i = 0
    files = db.select_files(cfg, limit)
    processed_dir = wikifilesdir+'-processed'
    for file in files:
        if i % 5 == 0:
            print (str(i)+ ' files processed')
        path = file[1]
        file_id = file[0]
        filepath = wikifilesdir+'/'+path
        with open(filepath, 'r', encoding='utf-8') as fh:
            datafile=fh.readlines()
            db.process_file(cfg, datafile, file_id)
        fh.close();
        move_file(path, wikifilesdir, processed_dir)
        i = i + 1
        
def get_tables_sizes():
    table_sizes = db.get_table_size(cfg)
    return table_sizes

def update_stats():
    snippets_size = ''
    engine = create_engine('postgresql://postgres@localhost:5432/sparsenlp')
    sql = "select b.id, b.cleaned_text as text from snippets b "
    sql += "where cleaned = 't'"
    data = pd.read_sql_query(sql, con=engine)
    #print (data.shape)
    word_counts = corp.get_word_counts_per_snippet(data, clean_text=False)
    freqs = corp.get_frequencies(word_counts)
    print (freqs['1960'])
    print (sorted(freqs.items(), key=operator.itemgetter(1), reverse=True)[:20])
    print (word_counts[5])
    vocabulary = corp.get_vocabulary(word_counts)
    print("O vocabulário tem %d palavras"% len(vocabulary) )
    table_sizes = get_tables_sizes()
    for row in table_sizes:
        if row[0] == 'snippets':
            snippets_size = row[1]
    db.update_words_vs_snippets(cfg, data.shape[0], len(vocabulary), snippets_size)
    
def process_snippets(_limit):
    #tokenizer for snippets 
    #500 snippets -> 30 min
    engine = create_engine('postgresql://postgres@localhost:5432/sparsenlp')
    sql = "select id, text from snippets"
    sql += "where length(text) > 100 and cleaned = 'f' and random() < 0.01 limit "+str(_limit)
    data = pd.read_sql_query(sql, con=engine)
    #print (data.shape)
    cleaned_texts = corp.clean_text(data, remove_stop_words=True, remove_punct=True, 
                   lemmas=True, remove_numbers=True, remove_spaces=True, remove_2letters_words=True, 
                   remove_apostrophe=True, method='spacy', spacy_disabled_components=['tagger', 'parser'])
    for text in cleaned_texts:
        print (text['id'])
        db.insert_cleaned_text(cfg, text['id'], text['snippet'])
        
def get_cleaned_data():
    conn_string = 'postgresql://postgres@localhost:5432/sparsenlp'
    engine = create_engine(conn_string)
    sql = "select id, cleaned_text as text from snippets where cleaned = 't'"
    dataframe = pd.read_sql_query(sql, con=engine)
    return dataframe
    

ERROR: <class 'FileNotFoundError'>


In [12]:
#register()
#process()
get_tables_sizes()
#process_snippets(10)
#update_stats()

[('files', '144 kB'),
 ('articles', '2264 kB'),
 ('news', '51 MB'),
 ('snippets', '576 MB'),
 ('words_vs_snippets', '8192 bytes'),
 ('chunks', '8192 bytes')]

In [85]:
sentence = "There is record of the French playwright Molière having attended many performances of the comédie italienne, or \"commedia dell\'arte\". He is even referenced in a performance by Angelo Costantini of his show \"Une Vie de Scaramouche\", which refers to the writer and poet. This might suggest that the servant character in many of Molière\'s plays, such as Dorine in his play \"Tartuffe\", might be based on this particular character archetype from the \"commedia dell'arte\"."
sentence = "The City of New York, often called New York City or simply New York, is the most populous city in the United States.[9] With an estimated 2017 population of 8,622,698[7] distributed over a land area of about 302.6 square miles (784 km2),[10][11] New York City is also the most densely populated major city in the United States.[12] Located at the southern tip of the state of New York, the city is the center of the New York metropolitan area, one of the most populous urban agglomerations in the world,[13][14] with an estimated 20.3 million people in its 2017 Metropolitan Statistical Area and 23.7 million residents in its Combined Statistical Area.[4][5] A global power city,[15] New York City has been described as the cultural, financial, and media capital[16][17] of the world,[18][19][20][21][22] and exerts a significant impact upon commerce,[22] entertainment, research, technology, education, politics, tourism, and sports. The city's fast pace[23][24] defines the term New York minute.[25] Home to the headquarters of the United Nations,[26] New York is an important center for international diplomacy."
sentence = "My name is   André, Silva and I like to practice wind   Surf and   Albert Einstein the   Rockefeller University and New York City"
#sentence = 'The United States of America are a great country'
#sentence = 'Apple is looking at buying U.K. startup for $1 billion'
sentence = "The cornell annual The budget for arXiv is approximately $826,000 for 2013 to 2017, funded jointly by Cornell University Library, the Simons Foundation (in both gift and challenge grant forms) and annual fee income from member institutions. This model arose in 2010, when Cornell sought to broaden the financial funding of the project by asking institutions to make annual voluntary contributions based on the amount of download usage by each institution. Annual donations were envisaged to vary in size between $2,300 to $4,000, based on each institution’s usage. , 174 institutions have pledged support for the period 2013–2017 on this basis, with a projected revenue from this source of approximately $340,000."

#cleaned_texts = corp.clean_text(sentence, remove_stop_words=True, remove_punct=True, lemmas=True, remove_numbers=True, remove_spaces=True, remove_2letters_words=True, remove_apostrophe=True, method='spacy', spacy_disabled_components=['tagger', 'parser'])
#print (cleaned_texts)



data = get_cleaned_data()
word_counts_per_snippet = corp.get_word_counts_per_snippet(data, clean_text=False)
print (word_counts_per_snippet[5])
freqs = corp.get_frequencies(word_counts_per_snippet)
print (freqs['america'])

snippets_by_word = corp.get_snippets_by_word(word_counts_per_snippet)
print (snippets_by_word['basketball'])
#print (snippets_by_word.keys())

snippets_by_word2 = corp.get_snippets_by_word2(data)
print (snippets_by_word2['basketball'])

Counter({'id': 987738, 'definition': 2, 'truck': 2, 'universally': 1, 'accepted': 1, 'suv': 1, 'government': 1, 'regulation': 1, 'simply': 1, 'category': 1, 'highway': 1, 'vehicle': 1, 'turn': 1, 'lump': 1, 'pickup': 1, 'minivan': 1, 'light': 1, 'auto': 1, 'industry': 1, 'settle': 1})
60
[{'snippet': 722146, 'counts': 1}, {'snippet': 722149, 'counts': 4}, {'snippet': 722145, 'counts': 2}, {'snippet': 722148, 'counts': 2}]
[722146, 722149, 722149, 722149, 722149, 722145, 722145, 722148, 722148]


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import safe_indexing

# setup SOM dimensions
H, W, N = 64, 64, 1000   # Network height, width and unit dimensions

df_train = get_cleaned_data()
#df_train = df_train.set_index("id", drop = True)

train_data = df_train.text

vectorizer = TfidfVectorizer(
                            lowercase=False,
                            #max_df=0.5, 
                            max_features=N,  
                            stop_words='english', 
                            use_idf=True)
X = vectorizer.fit_transform(train_data)

#X = safe_indexing(X, df_train.index.values)
#x = X.data 
#csr_matrix((X.data, df_train.index.values, indptr), shape=(3, 3)).toarray()

print (type(X))
print (X.shape)
#print(vectorizer.get_feature_names())

# setup SOM network
som = Som(H, W, N, topology.RECT, verbose=True) # , verbose=True
# reinit the codebook (not needed)
som.codebook = np.random.rand(H, W, N).astype(som.codebook.dtype, copy=False)

# train the SOM
som.train(X)
print (type(som.codebook))

#serializing objects 
np.save('./codebook.npy', som.codebook)
df_train.to_pickle('./dataframe.pkl')
np.savez('./x.npz', data=X.data, indices=X.indices, indptr=X.indptr, shape=X.shape)


#X[5]
#print (df_train.text[578553])
#bmus = som.bmus(X[1])
#print (bmus)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


<class 'scipy.sparse.csr.csr_matrix'>
(3520, 1000)
sorting indices
<class 'numpy.ndarray'>


In [56]:
codebook = np.load('./codebook.npy')
dataframe = pd.read_pickle('./dataframe.pkl')
loader = np.load('./x.npz')

X = csr_matrix((loader['data'], loader['indices'], loader['indptr']), shape=loader['shape'])

som = Som(H, W, N, topology.RECT, verbose=True) # , verbose=True
som.codebook = codebook

word_counts_per_snippet = corp.get_word_counts_per_snippet(dataframe, clean_text=False)
snippets_by_word = corp.get_snippets_by_word(word_counts_per_snippet)

i = 0
for key, value in snippets_by_word.items():
    if i < 5:
        if key != 'id':
            a = np.zeros((H, W), dtype=np.int)
            print ('########  ' +str(key)+ '  ########')
            for snippet_count in value:
                idx =  df_train.index[df_train['id'] == snippet_count['snippet']].tolist()[0]
                bmus = som.bmus(X[idx])
                #print (bmus[0].tolist())
                a[bmus[0][0], bmus[0][1]] = snippet_count['counts']
                
            im = Image.fromarray(a)
            im.save("./images/"+key+".png")  
    else:
        break
    i += 1

########  continental  ########
########  europe  ########
########  term  ########
########  school  ########


In [6]:
codebook = np.load('./codebook.npy')
dataframe = pd.read_pickle('./dataframe.pkl')
loader = np.load('./x.npz')

X = csr_matrix((loader['data'], loader['indices'], loader['indptr']), shape=loader['shape'])
print (X.shape)
word = 'cat'
finger.create_fingerprint(word, dataframe, codebook, X, H, W, 'binary')

(3520, 1000)
########  cat  ########


<h3>Export snippets to csv file</h3>

In [52]:
current_chunk_id = db.get_current_chunk_id(cfg)[0][0]
chunck_size = 10000
total_chunck_size = 55000

chunck_size = 100
total_chunck_size = 100


engine = create_engine('postgresql://postgres@localhost:5432/sparsenlp')
sql = "select id, text from snippets where id >= "+str(current_chunk_id)+" and length(text) > 100 order by id limit "+str(total_chunck_size)
df = pd.read_sql_query(sql, con=engine)

current = 0
while current < total_chunck_size + 1:
    
    df_chunk = df[current: current + chunck_size]
    df_chunk_min = df_chunk['id'].min()
    df_chunk_max = df_chunk['id'].max()
    #chunk_filename = 'raw_minid_maxid_size
    chunk_filename = 'raw_'+str(df_chunk_min)+'_'+str(df_chunk_max)+'_'+str(df_chunk.shape[0])
    df_chunk.to_csv('./chuncks/new/'+chunk_filename+'.bz2', index=False, compression='bz2', columns=['id', 'text'], encoding='utf-8')
    #print (df_chunk.shape, df_chunk_min, df_chunk_max, chunk_filename)
    #db.create_chunk(cfg, df_chunk_min, df_chunk_max, df_chunk.shape[0])
    current += chunck_size

5779858